In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as pp
import xml.etree.ElementTree as ET
import pandas as pd
import os, sys, glob
import subprocess
import re
import scipy as sp
import sympy as sy
pp.rcParams.update({
    "text.usetex": False,
})
from IPython.display import display, Markdown, Latex
from sympy import init_printing
from cycler import cycler
EP = '{http://www.molpro.net/schema/molpro-output}' #element prefix, somehow gets added
%matplotlib widget

## Display and pretty printing function

In [ ]:
def md(*args, **kwargs):
    return display(Markdown(*args, **kwargs))

def latex(*args, **kwargs):
    return display(Latex(*args, **kwargs))

## Functions for Generating Wavefunction as function of x

In [ ]:
def slater(beta, xdata):
    ydata = np.exp(-beta * np.abs(xdata))
    return  ydata

def f12(beta, xdata):
    return (1/beta) * (1 - slater(beta, xdata))
    
def get_gaussians(alphas, coeffs, x):
    '''Returns sum of gaussians of x evaluated from alphas and coeffs'''
   
    fx = np.zeros_like(x)
    for n, alph in enumerate(alphas):
        c = coeffs[n]
        fx += c * np.exp(-alph * x**2)

    return fx
    
def get_alpha_range(min_alpha, max_alpha, n_gem):
    '''Returns an array of alphas based on the formula below'''
    return np.array([min_alpha * (max_alpha / min_alpha) ** (i/(n_gem-1)) for i in range(n_gem)])

## Functions to extract Alphas and Coeffs from Molpro output file

In [ ]:
search_str = " Geminal optimization for beta="
def get_coefficients_from_output(output_fname):
    #with open(output_fname, 'r') as out:

    num_lines = 7
    
    command = f"grep -A{num_lines} '{search_str}' {output_fname}"
    lines = subprocess.check_output(command, shell=True)
    lines = lines.decode('utf-8').split('--\n')

    return lines 

def lines_to_dict(lines):

    data = dict()
    for l in lines:
        if "NO CONVERGENCE IN GEMINAL FIT" in l:
            continue
        rows = l.split('\n')
        for row in rows:
            if search_str in row:
                beta = float(row.lstrip(search_str))
                continue
            if 'Weight function' in row:
                m = float(re.split('m=|,', row)[1])
                omega = float(re.split('omega= ', row)[1])
                continue
            if 'Alpha' in row:
                string_alphas = row.split()[1:]
                alphas = [float(a) for a in string_alphas]
                continue
            if 'Coeff' in row:
                string_coeffs = row.split()[1:]
                coeffs = [float(a) for a in string_coeffs]
                continue

        data[beta] = dict(
                m = m,
                omega = omega,
                alphas = alphas,
                coeffs = coeffs,
            )
    return data
    #return coefficients



In [ ]:
molpro_out = 'betas_for_H.out'
lines = get_coefficients_from_output(molpro_out)
data = lines_to_dict(lines)


In [ ]:
def combine_betas(beta1, beta2):
    return round((beta1 + beta2) / 2, 2)
    
def write_expfile(beta1, beta2, fname):
    beta3 = combine_betas(beta1, beta2)
    file_content = []
    file_content.append(
        f"# gamma_1 = {beta1:4.2f}; " + \
        f"gamma_2 = {beta2:4.2f}; gamma_3 = {beta3:4.2f}"
    )
    file_content.append("# gamma_index type_of_data data")
    
    for n, beta in enumerate([beta1, beta2, beta3]):
        i = n + 1
        dat = data[beta]
        alphas = dat['alphas']
        coeffs = dat['coeffs']

        file_content.append(f"{i} nCo 1")
        file_content.append(f"{i} Exp {','.join((str(a) for a in alphas))}")
        file_content.append(f"{i} Coe {','.join((str(a) for a in coeffs))}")
        
    with open(fname, 'w') as out:
        out.write('\n'.join(file_content) + '\n')

# gamma_index type_of_data data
#1 nCo 1
#1 Exp 0.19532,0.81920,2.85917,9.50073,35.69989,197.79328
#1 Coe -0.27070,-0.30552,-0.18297,-0.10986,-0.06810,-0.04224

In [ ]:
beta1 = 1.3
beta2 = 1.3
beta3 = combine_betas(beta1, beta2)
fname = f"expfile_{beta1:4.2f}_{beta2:4.2f}_{beta3:4.2f}.txt"
print(fname)
write_expfile(beta1, beta2, fname)

## alpha range function

In [ ]:
get_alpha_range(0.5, 500, 6)

## Optimization functions
The weights are from Werner paper

In [ ]:
def get_weights(x, beta):
    '''Returns x * W(x)'''
    W = np.exp(-0.5 * (np.pi * beta ** 2) ** (1/3) * x)
    return W * x

def weightify(func, beta):
    '''Returns F, where F(x) = x * W(x) * f(x)'''
    def weighted_func(x, *args, **kwargs):
        weights = get_weights(x, beta)
        return weights * func(x, *args, **kwargs)
    return weighted_func

def obj_func(x, yvalues, ref_yvalues):
    '''Returns sum(y^2 - y_ref^2) 
    !! x is not used here !!'''
    residuals = yvalues - ref_yvalues
    return np.sum(residuals**2)

def get_rss_manually(x, data):
    '''Return dict(beta: obj_func()) for each beta in data
        Structure of data <dict>:
            'beta' : <float>
            'alphas' : <array of floats>
            'coeffs' : <array of floats>
    '''
    rsss = dict()
    for n, dat in enumerate(data):
        beta = dat['beta']
        alphas = dat['alphas']
        coeffs = dat['coeffs']

        weights = get_weights(x, beta)

        yfunc = fixed_alpha_gaussians(alphas)
        yvalues = weights * yfunc(x, *coeffs)
        
        
        ref_yvalues = weights * slater(x, beta)

        rss = obj_func(x, yvalues, ref_yvalues)

        rsss[beta] = rss

    return rsss

def optimize_coeffs(beta, alpha_set0, *args, xdata=None, get_rss=False, **kwargs):
    '''
    For given beta, and starting alpha_set0, find optimized coeffs and alphas.
    '''

# === From other notebook, for reference only ===

In [ ]:
        
        
def minimized_coeffs(beta, alpha_set, *args, xdata=None, get_rss=False, **kwargs):
    '''
    For given beta and alpha_set, find optimized coeffs.
    '''
    if xdata is None:
        xdata = np.linspace(0, 4, 1000)
    ydata = slater(beta, xdata)
    weights = get_weights(xdata, beta)
    
    weighted_ydata = weights * ydata

    fixed_alpha = fixed_alpha_gaussians(alpha_set)
    weighted_fixed_alpha = weightify(fixed_alpha, beta)
    
    opt = sp.optimize.curve_fit(
        weighted_fixed_alpha, xdata, weighted_ydata, 
        *args,
        p0 = np.zeros_like(alpha_set),  
        **kwargs
        #bounds=(-inf, inf), 
    ) 

    popt = opt[0]
    popc = opt[1]
    
    if get_rss:
        residuals = weighted_ydata - weighted_fixed_alpha(xdata, *popt)
        rss = np.sum(residuals**2)
        return opt + (rss,)
    else:
        return opt
    

def get_minimized_coeffs_table(betas, alpha_set):
    beta_alpha_dict = {}
    data = []
    rss_col = []
    for n, bet in enumerate(betas):
        popt, pcov, rss = minimized_coeffs(bet, alpha_set, get_rss=True)
        data.append(popt)
        rss_col.append(rss)

    data = np.array(data)
    df = pd.DataFrame(data=data, columns=alpha_set)
    df.insert(0, r'$\beta$', betas)
    df['rss'] = rss_col
    return df
    
def transform_coeff_table_to_data(coeffs_table):
    alpha_set = coeffs_table.columns.values[1:-1]
    betas = coeffs_table[r'$\beta$']#.values
    data = []
    for n, bet in enumerate(betas):
        dic = {}
        dic['beta'] = bet
        dic['alphas'] = alpha_set
        dic['coeffs'] = coeffs_table.iloc[n].values[1:-1]
        data.append(dic)

    return data


def transform_data_to_alpha_coeff_tables(data):
    coeff_dict = {r'$\beta$' : []}
    alpha_dict = {r'$\beta$' : []}

    for n, dat in enumerate(data):
        alpha_dict[r'$\beta$'].append(dat['beta'])
        coeff_dict[r'$\beta$'].append(dat['beta'])
        
        for i, alph in enumerate(dat['alphas']):
            c = dat['coeffs'][i]
            
            if n == 0:
                alpha_dict[fr"$\alpha_{i}$"] = []
                coeff_dict[fr"$c_{i}$"] = []

            alpha_dict[fr"$\alpha_{i}$"].append(alph)
            coeff_dict[fr"$c_{i}$"].append(c)

    return pd.DataFrame(alpha_dict), pd.DataFrame(coeff_dict)

  

## Plotting Functions

In [ ]:
def plot_single_and_compare(x, fx, bet, ax, *args, **kwargs):
    ax.plot(x, fx, label=fr"$\beta = {bet:4.2f}$", *args, **kwargs)
    ax.plot(x, slater(bet, x), linestyle = '--', *args, **kwargs)


def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=100):
    new_cmap = mpl.colors.LinearSegmentedColormap.from_list(
        'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
        cmap(np.linspace(minval, maxval, n)))
    return new_cmap

def plot_in_range(x, data, ax):
    '''Argument "data" must be a dictionary that contains keywords `beta`, `alphas`, and `coeffs`'''
    betas = [dat['beta'] for dat in data]
    norm = mpl.colors.Normalize(vmin=min(betas), vmax=max(betas))
    cmap = truncate_colormap(pp.cm.viridis, minval= 0.05, maxval=0.95)
    betas = []
    for n, dat in enumerate(data):

        bet = dat['beta']
        if dat['beta'] in betas: continue
        betas.append(bet)
        fx = get_gaussians(dat['alphas'], dat['coeffs'], x)

        color=cmap(norm(bet))
        plot_single_and_compare(x, fx, bet, ax, color=color)


    ax.set_xlabel(r"$r_{12}$")
    ax.set_ylabel(r"$f(r_{12})$")
    ax.set_xlim(min(x), max(x))
    ylims = ax.get_ylim()
    if ylims[0] < 0:
        ax.set_ylim(0, None)

In [ ]:
fig, ax = pp.subplots()
x = np.linspace(0, 4.0, 1000)
betas = (0.8, 1.25, 1.70, 2.20) #[1] # [1] for WATOC
norm = mpl.colors.Normalize(vmin=min(betas), vmax=max(betas))
cmap = truncate_colormap(pp.cm.viridis, minval= 0.05, maxval=0.95)

for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(2)
    
for bet in betas:
    color = cmap(norm(bet))
    ax.plot(
        x, f12(bet, x), 
        color=color, label=fr"$\gamma={bet}$",
        linewidth = 3
    )

ax.legend()
label_fs = 24
ax.set_xlim(0,4)
ax.set_ylim(0, 1/min(betas)+0.01)
ax.set_xlabel(r"$r_{12}$", fontsize=label_fs)
ax.set_ylabel(r"$f(r_{12})$", fontsize=label_fs)
fig.set_size_inches(8,5)
fig.tight_layout()

# Fort just one function

In [ ]:
fig, ax = pp.subplots()
x = np.linspace(0, 4.0, 1000)
betas = [1]

for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(2)
    
for bet in betas:
    color = 'k'
    ax.plot(
        x, f12(bet, x), 
        color=color, label=fr"$\gamma={bet}$",
        linewidth = 3
    )
ax.plot(x, x, linestyle='--', color=color)
#ax.legend()
label_fs = 24
ax.set_xlim(0,4)

ax.set_ylim(0, 1/min(betas)+0.01)
#ax.set_xlabel(r"$r_{12}$", fontsize=label_fs)
#ax.set_ylabel(r"$f(r_{12})$", fontsize=label_fs)
ax.set_xticks([])
ax.set_yticks([])
fig.set_size_inches(8,5)
fig.tight_layout()

In [ ]:
pwd

In [ ]:
fig.savefig('f12_vs_r.svg', transparent=True)

## Analysizing slater and f12 together

In [ ]:
fig, axes = pp.subplots(2)
x = np.linspace(0, 4.0, 1000)
betas = (0.8, 1.25, 1.70, 2.20) #[1] # [1] for WATOC
norm = mpl.colors.Normalize(vmin=min(betas), vmax=max(betas))
cmap = truncate_colormap(pp.cm.viridis, minval= 0.05, maxval=0.95)

funcs_to_plot = [
    lambda bet, x: bet * f12(bet, x),
    slater
]


for n, ax in enumerate(axes):
    
    for axis in ['top','bottom','left','right']:
        ax.spines[axis].set_linewidth(2)

    func = funcs_to_plot[n]
    for bet in betas:
        color = cmap(norm(bet))
        ax.plot(
            x, func(bet, x), 
            color=color, label=fr"$\gamma={bet}$",
            linewidth = 3
        )


    ax.set_xlim(0,4)
    ax.set_ylim(0,1.01)
    #ax.set_xlabel(r"$e^- - e^-$ distance, $r_{12}$", fontsize=label_fs)
    #ax.set_ylabel(r"$f_{12}(r_{12})$", fontsize=label_fs)
    fig.set_size_inches(8,5)
    fig.tight_layout()

ax.legend()
label_fs = 24

## Now to plot and compare

In [ ]:
df_alpha, df_c = transform_data_to_alpha_coeff_tables(data)

md("## Default alphas and coefficients")

rss = get_rss_manually(np.linspace(0,4,1000), data)

add_df = pd.DataFrame(
    {df_alpha.columns[0] : rss.keys(), 'rss' : rss.values()}
)
display(add_df)
df_alpha = df_alpha.merge(add_df, how='inner', on=add_df.columns[0])
df_c = df_c.merge(add_df, how='inner', on=add_df.columns[0])
display(df_alpha)
display(df_c)

md("### Latex output")
print(df_c.to_latex()) 



# Alpha vs $\beta$ relationship is almost linear

In [ ]:
betas = []
alphas_by_betas = [[] for _ in range(6)]
coeffs_by_betas = [[] for _ in range(6)]

for dat in data:
    bet = dat['beta']
    if bet in betas: continue
    betas.append(bet)
    for n, alph in enumerate(dat['alphas']):
        alphas_by_betas[n].append(alph)
        coeffs_by_betas[n].append(dat['coeffs'][n])
    

alphas_by_betas = np.array(alphas_by_betas)
print(alphas_by_betas)

fig, ax = pp.subplots()
ax.plot((betas[0],betas[-1]), (0, 1), color='k', linestyle='--')
labels = [fr"$\alpha_{i}$" for i in range(6)]
for i, alpha in enumerate(alphas_by_betas):
    y = (alpha - alpha[0]) / (alpha[-1] - alpha[0]) 
    ax.plot(betas, y, label=labels[i])

ax.legend()
ax.set_xlabel(r"$\beta$")
ax.set_ylabel(r"Normalized $\alpha$")

cols = [fr'$\alpha_{i}$' for i in range(1, 7)]
df = pd.DataFrame(data=alphas_by_betas.T, columns=cols, index=betas)
print(df.to_latex())

In [ ]:
md("### Comparing psis of default molpro optimization")
fig, axes = pp.subplots(2,1)
fig.set_size_inches(8, 10)


plot_in_range(np.linspace(0,4, 1000), data, axes[0])
plot_in_range(np.linspace(0,0.1, 1000), data, axes[1])
axes[0].legend()

# Minimizing with Fixed Alphas

In [ ]:
alpha_sets = dict(
    alpha_set_0 = get_alpha_range(0.14, 444, 6),
    alpha_set_1 = get_alpha_range(0.07, 222, 6),
    alpha_set_2 = get_alpha_range(0.10, 555, 6),
)

In [ ]:
alpha_label = 'alpha_set_0'
bet = 0.8

alpha_set = alpha_sets[alpha_label]
md(fr"## Alpha Set = `{alpha_label}` and $\beta = {bet}$")

fig, axes = pp.subplots(2, 1)

popt, pcov = minimized_coeffs(bet, alpha_set)

x = np.linspace(0, 4, 1000)
y = get_gaussians(alpha_set, popt, x)
plot_single_and_compare(x, y, bet, axes[0])

x = np.linspace(0, 0.1, 1000)
y = get_gaussians(alpha_set, popt, x)
plot_single_and_compare(x, y, bet, axes[1])

In [ ]:
alpha_label = 'alpha_set_0'
alpha_set = alpha_sets[alpha_label]

md(fr"## Comparing fitted Gaussians to true function for `{alpha_label}`")

fig, axes = pp.subplots(2, 1)
fig.set_size_inches(8, 10)
x = np.linspace(0, 4)

#betas = np.linspace(0.6, 3.2, 15)
#betas = np.arange(0.72, 0.8, 0.02) # Low range
betas = np.linspace(0.76, 2.6, 5) # extended range
betas = [0.8, 1.25, 1.70, 2.20] # Werner et al range
#betas = np.arange(2.3, 3.0, 0.2) # High range


coeffs_table = get_minimized_coeffs_table(betas, alpha_set)
minimized_data = transform_coeff_table_to_data(coeffs_table)

display(coeffs_table)
display(minimized_data)

plot_in_range(x, minimized_data, axes[0])

axes[0].legend()
x = np.linspace(0, 0.1)

plot_in_range(x, minimized_data, axes[1])

md("### Table of coefficients")
display(coeffs_table)
print(coeffs_table.to_latex())

## Comparing fixed alpha with molpro

In [ ]:
comp_df = df_alpha[[r'$\beta$', 'rss']].merge(coeffs_table[[r'$\beta$', 'rss']], how='inner', on=r'$\beta$')
comp_df.columns = [r'$\beta$', 'rss_default', 'rss_alpha_set_0']
display(comp_df)
print(comp_df.to_latex())

In [ ]:
alphabet_table = get_coeffs(betas, alpha_set)

In [ ]:
print(alphabet_table.to_csv())